In [17]:
import QuantLib as ql
import numpy as np

# ------------------- market inputs -------------------------------
spot = 100.0
r_flat, q_flat = 0.02, 0.00
today = ql.Date(10, 7, 2025)
ql.Settings.instance().evaluationDate = today
dc = ql.Actual365Fixed()
calendar   = ql.NullCalendar()

rTS  = ql.YieldTermStructureHandle(ql.FlatForward(today, r_flat, dc))
qTS  = ql.YieldTermStructureHandle(ql.FlatForward(today, q_flat, dc))
spotH = ql.QuoteHandle(ql.SimpleQuote(spot))

# scattered option quotes:  (strike, maturity in years, mid-price, type)
quotes = [
    ( 90.0, 0.25, 11.85, ql.Option.Call),
    (100.0, 0.25,  6.90, ql.Option.Call),
    (110.0, 0.25,  3.29, ql.Option.Call),
    #( 95.0, 1.00, 12.20, ql.Option.Put ),
    #(105.0, 1.00, 15.10, ql.Option.Put ),
    #(120.0, 1.50, 24.65, ql.Option.Put ),
]



# ------------------- build the CalibrationSet --------------------
calib = ql.CalibrationSet()
bs_vols = []
for K, T, price, optType in quotes:
    payoff   = ql.PlainVanillaPayoff(optType, K)
    expiry   = today + int(T*365 + 0.5)
    exercise = ql.EuropeanExercise(expiry)
    option   = ql.VanillaOption(payoff, exercise)


    # Calculate BS implied volatility
    process = ql.BlackScholesMertonProcess(spotH, qTS, rTS, ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, calendar, 0.02, dc)))

    try:
        implied_vol = option.impliedVolatility(price, process, 1e-6, 100, 0.001, 2.0)
        bs_vols.append((K, T, price, optType, implied_vol))
        print(f"BS vol  K={K:6.1f}  T={T:.4f}  σ={implied_vol:.3f}")
    except RuntimeError as e:
        print(f"impliedVol failed  K={K:.1f}  T={T:.4f}: {e}")
        bs_vols.append((K, T, price, optType, np.nan))


    calib.push_back((option, ql.SimpleQuote(implied_vol)))

# ------------------- Andreasen-Huge surface ----------------------
ah_interp = ql.AndreasenHugeVolatilityInterpl(
    calib,                         # scattered quotes
    spotH, rTS, qTS,
    ql.AndreasenHugeVolatilityInterpl.CubicSpline,
    #ql.AndreasenHugeVolatilityInterpl.CallPut   # <-- quotes are prices
    ql.AndreasenHugeVolatilityInterpl.Call
)

ah_surface = ql.AndreasenHugeVolatilityAdapter(ah_interp)
ah_surface.enableExtrapolation()
volTS = ql.BlackVolTermStructureHandle(ah_surface)

# quick test -------------------------------------------------------
print("ATM vol at 0.25y =", volTS.blackVol(0.25, spot))

BS vol  K=  90.0  T=0.2500  σ=0.264
BS vol  K= 100.0  T=0.2500  σ=0.335
BS vol  K= 110.0  T=0.2500  σ=0.338
ATM vol at 0.25y = 0.33479691934836253
